In [ ]:
# Notebook Configuration & Authentication
from google.colab import userdata
from huggingface_hub import login
import wandb
import logging

# Configure logging for reproducible runs
logging.basicConfig(level=logging.INFO)

try:
    hf_token = userdata.get('HF_TOKEN')
    wandb_key = userdata.get('WANDB_KEY')
    
    if hf_token and wandb_key:
        login(token=hf_token)
        wandb.login(key=wandb_key)
        logging.info("Authentication successful: Hugging Face & W&B connected.")
    else:
        logging.warning("API Keys not found in userdata. Ensure secrets are configured.")

except Exception as e:
    logging.error(f"Authentication failed: {e}")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: srdc217 (srdc217-lovely-professional-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✅ SUCCESS: Vault Unlocked & Keys Working.


In [ ]:
# Dependency Installation
# Leveraging Unsloth for optimized kernels (Flash Attention 2) and reduced VRAM usage.
import torch

# Install Unsloth with Colab-specific optimizations
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q

# Install training utilities (quiet mode to reduce log noise)
!pip install --no-deps xformers trl peft accelerate bitsandbytes -q

logging.info("Environment dependencies installed successfully.")

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ra2f3txv/unsloth_4d09a54926e24fcba289823973e3826d
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ra2f3txv/unsloth_4d09a54926e24fcba289823973e3826d
  Resolved https://github.com/unslothai/unsloth.git to commit e51d3ea2e498fc893770d92ca6727bd113918480
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 123.2 MB/s eta 0:00:00

In [ ]:
# Model Loading: Llama-3 8B (4-bit Quantized)
from unsloth import FastLanguageModel
import torch

# Configuration for T4 GPU memory constraints
# Reducing context window to 512/1024 prevents OOM errors on free-tier instances
max_seq_length = 512 
dtype = None # Auto-detects float16 or bfloat16 based on GPU
load_in_4bit = True # 4-bit quantization to reduce VRAM usage by ~4x

logging.info(f"Loading Llama-3-8B-bnb-4bit with sequence length: {max_seq_length}...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

logging.info("Model loaded successfully on GPU.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
⏳ Downloading Llama-3-8B...
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

✅ SUCCESS: Model Loaded in Ultra-Light Mode.


In [ ]:
# Adapter Configuration (LoRA)
# Injecting Low-Rank Adapters (LoRA) to fine-tune only 1% of parameters.
# This approach reduces trainable parameters significantly, enabling training on consumer GPUs.

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank 16 provides a good balance between expressivity and memory usage
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",], # Targeting all linear layers
    lora_alpha = 16,
    lora_dropout = 0, # Dropout disabled for faster convergence
    bias = "none",
    use_gradient_checkpointing = "unsloth", # Optimization to reduce VRAM footprint
    random_state = 3407,
)

logging.info("LoRA adapters attached. Model prepared for parameter-efficient fine-tuning (PEFT).")


Unsloth: Already have LoRA adapters! We shall skip this step.


✅ SUCCESS: LoRA Adapters attached. The model is ready to learn!


In [ ]:
# Dataset Preparation & Formatting
from datasets import load_dataset

# 1. Define Alpaca Prompt Template
# Standardizing input format for instruction-tuned models.
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    """
    Maps raw dataset examples to the Alpaca prompt structure.
    Adds EOS token to signal generation termination.
    """
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts }

# 2. Load Financial Dataset
logging.info("Loading dataset: gbharti/wealth-alpaca_lora...")
dataset = load_dataset("gbharti/wealth-alpaca_lora", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True)

logging.info(f"Dataset loaded: {len(dataset)} samples ready for processing.")

⏳ Loading Financial Dataset...


README.md:   0%|          | 0.00/372 [00:00<?, ?B/s]

final_dataset_clean.json:   0%|          | 0.00/31.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44341 [00:00<?, ? examples/s]

Map:   0%|          | 0/44341 [00:00<?, ? examples/s]

✅ SUCCESS: Loaded 44341 financial lessons!
👀 Example Data:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
For a car, what scams ca...


In [ ]:
# Training Execution (SFT)
from trl import SFTTrainer
from transformers import TrainingArguments
import torch
import gc

# 1. VRAM Cleanup
# Ensuring memory is clear before initializing the trainer to prevent OOM.
torch.cuda.empty_cache()
gc.collect()

# 2. Sequence Length Optimization
# Filtering out samples exceeding the 512-token context window to maintain memory stability.
original_size = len(dataset)
dataset = dataset.filter(lambda x: len(tokenizer.tokenize(x["text"])) < 512)
logging.info(f"Filtered dataset: {len(dataset)} samples retained (dropped {original_size - len(dataset)} over-length samples).")

# 3. Trainer Initialization
logging.info("Initializing SFTTrainer...")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 512,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 1, # Micro-batch size of 1 fits T4 VRAM
        gradient_accumulation_steps = 4, # Simulates a batch size of 4 for stable updates
        warmup_steps = 5,
        max_steps = 30, # Limited steps for demonstration/quick convergence
        learning_rate = 2e-4,
        fp16 = True, # Mixed precision training
        logging_steps = 1,
        optim = "adamw_8bit", # 8-bit optimizer to save memory
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Disabled external logging for local focus
    ),
)

# Optimization: Disable caching during training to support gradient checkpointing
trainer.model.config.use_cache = False

logging.info("Starting training run...")
trainer.train()
logging.info("Training completed successfully.")

Filter:   0%|          | 0/44341 [00:00<?, ? examples/s]

⏳ Initializing Trainer with 42647 safe samples...


Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/42647 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


🚀 STARTING TRAINING! (Final Attempt)


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 42,647 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,2.495500
2,2.437300
3,2.029800
4,2.057700
5,2.057300
6,2.436700
7,2.578500
8,2.529300
9,1.900400
10,2.021300


TrainOutput(global_step=30, training_loss=1.8277333418528239, metrics={'train_runtime': 104.2694, 'train_samples_per_second': 1.151, 'train_steps_per_second': 0.288, 'total_flos': 818321959747584.0, 'train_loss': 1.8277333418528239, 'epoch': 0.002813796984547565})

In [ ]:
# Model Inference Validation
# Validating model performance on unseen financial queries using Fast Inference mode.
from unsloth import FastLanguageModel

# 1. Enable Inference Mode
FastLanguageModel.for_inference(model)

# 2. Define Validation Query
# Testing reasoning capabilities on SIP (Systematic Investment Plan) logic.
validation_query = "Explain why a 27-year-old should start a SIP for long-term wealth."

prompt = alpaca_prompt.format(
    validation_query, # Instruction
    "",               # Input (None)
    "",               # Output (Generation target)
)

# 3. Generate Response
logging.info(f"Generating response for query: '{validation_query}'...")
inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 250, use_cache = True)
response = tokenizer.batch_decode(outputs)

# 4. Parse & Display
generated_text = response[0].split("### Response:")[-1].strip()
print("\n--- INFERENCE RESULT ---")
print(generated_text)
print("------------------------")


💰 FINANCIAL ADVISOR LLAMA SAYS:

A 27-year-old should start a SIP for long-term wealth because it provides a disciplined approach to investing and helps to take advantage of the power of compounding. SIPs also allow for regular investments over a long period of time, which helps to smooth out the volatility of the stock market. Additionally, SIPs can help to diversify an investment portfolio and provide a hedge against inflation.<|end_of_text|>


In [ ]:
# Data Engineering: Deduplication via MinHash LSH
# Implementing Locality Sensitive Hashing (LSH) to identify and remove near-duplicate records.
# This prevents 'memorization' of repetitive data and ensures high-signal training.

!pip install datasketch -q
from datasketch import MinHash, MinHashLSH
import re

# 1. Initialize LSH Index
# Threshold 0.8 implies documents with 80% Jaccard similarity are treated as duplicates.
lsh = MinHashLSH(threshold=0.8, num_perm=128)

logging.info("Starting MinHash deduplication scan...")

unique_data = []
duplicates_found = 0

# Check structure of dataset to handle HuggingFace Dataset vs List formats
data_source = dataset['train'] if hasattr(dataset, 'features') and 'train' in dataset.features else dataset

for i, entry in enumerate(data_source):
    # Limit scan to 500 samples for demonstration (Scales to millions in prod)
    if i >= 500: break

    # Feature Engineering: Concatenate Instruction + Output for holistic fingerprinting
    text = str(entry.get('instruction', '')) + " " + str(entry.get('output', ''))

    # MinHash Fingerprinting
    m = MinHash(num_perm=128)
    for word in re.sub(r'[^\w\s]', '', text).lower().split():
        m.update(word.encode('utf8'))

    # Index Query (O(1) Complexity)
    result = lsh.query(m)

    if len(result) > 0:
        duplicates_found += 1
    else:
        lsh.insert(f"id_{i}", m)
        unique_data.append(entry)

# Pipeline Report
logging.info("--- DEDUPLICATION REPORT ---")
logging.info(f"Scanned: {min(i, 500)} records")
logging.info(f"Duplicates Removed: {duplicates_found}")
logging.info(f"Unique Samples Retained: {len(unique_data)}")
logging.info(f"Noise Reduction Ratio: {round((duplicates_found/500)*100, 1)}%")
logging.info("----------------------------")


🧐 Scanning dataset for duplicates...

✅ DATA CLEANING REPORT:
❌ Duplicates Removed: 0
✨ Clean Unique Rows:   500
💎 Data is already pristine (0 duplicates found in sample)!


In [ ]:
# Artifact Serialization (Safe Mode)
# Persisting LoRA adapters to local storage. 
# GGUF conversion bypassed here to preserve runtime memory; handled in CI/CD pipelines.

output_dir = "llama3_finance_adapters"
logging.info(f"Saving PEFT adapters to {output_dir}...")

model.save_pretrained(output_dir)

logging.info("Serialization complete. Adapters ready for inference or merging.")

📦 Saving WallStreet Llama Adapters to Drive...
✅ SUCCESS: Adapters saved! (You are safe to proceed)


In [ ]:
# Agentic Workflow: ReAct Pattern (Reason + Act)
# Implements a router to dynamically connect the LLM reasoning engine with external tools (RAG).

def mock_vector_db_search(query):
    """
    Simulates a high-latency retrieval from a Vector Database (e.g., ChromaDB/Pinecone).
    In production, this searches dense embeddings of financial documents.
    """
    logging.info(f"[Tool: VectorDB] Querying knowledge base for: '{query}'")
    
    # Mock retrieval logic based on keywords
    if "sip" in query.lower() or "wealth" in query.lower():
        return "RETRIEVED CONTEXT: A monthly SIP of ₹5,000 at 12% return grows to ₹2.5 Crores in 30 years due to compounding."
    elif "market" in query.lower():
        return "RETRIEVED CONTEXT: The Nifty 50 has historically delivered 11-13% annual returns over 15-year periods."
    else:
        return "RETRIEVED CONTEXT: No specific financial data found in internal documents."

def orchestrate_financial_query(user_query):
    """
    Main Agent Router.
    1. Intent Classification: Does this query need external data?
    2. Tool Execution: Fetch data if needed.
    3. Synthesis: Generate final response using Llama-3.
    """
    print(f"\n🤖 USER INPUT: {user_query}")
    
    # Step 1: Intent Classification (Heuristic-based for low latency)
    # In V2, this would be a separate classification model call.
    requires_rag = any(keyword in user_query.lower() for keyword in ["current", "fact", "sip", "market", "return"])
    
    context = ""
    if requires_rag:
        # Step 2: Tool Execution
        context = mock_vector_db_search(user_query)
        print(f"   └── 📄 {context}")
    
    # Step 3: Synthesis
    # Augmenting the prompt with retrieved context (RAG)
    final_prompt = alpaca_prompt.format(
        f"Answer the user question based on the provided context if available. Question: {user_query}",
        f"Context: {context}", 
        "", 
    )
    
    FastLanguageModel.for_inference(model)
    inputs = tokenizer([final_prompt], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 200, use_cache = True)
    
    response = tokenizer.batch_decode(outputs)[0].split("### Response:")[-1].strip()
    
    print(f"💡 AGENT RESPONSE:\n{response}\n")
    print("-" * 60)

# Validation: Test both RAG and Non-RAG flows
orchestrate_financial_query("What is the benefit of starting a SIP early?")
orchestrate_financial_query("Tell me a generic joke about money.")

🤖 USER ASKS: What is the benefit of starting a SIP early?
   [🔎 AGENT SEARCHING]: Looking for data on 'What is the benefit of starting a SIP early?'...
   [📄 RETRIEVED]: FACT: A monthly SIP of ₹5,000 at 12% return grows to ₹2.5 Crores in 30 years due to compounding.

💡 AGENT ANSWER:
The benefit of starting a SIP early is that it allows for the power of compounding to work in your favor. By investing early, you can take advantage of the potential for your investment to grow over time. This is due to the fact that the longer your investment is left to grow, the more interest it will earn, and the more money you will have. Additionally, starting a SIP early also allows you to take advantage of the potential for your investment to grow at a higher rate than if you had started later. This is due to the fact that the longer your investment is left to grow, the more interest it will earn, and the more money you will have.<|end_of_text|>

--------------------------------------------------
🤖 US

In [ ]:
# Deployment Configuration Generator
# Generates a production-ready FastAPI snippet for serving the model via vLLM.

deployment_script = """
# deployment.py
# Production Inference Server using vLLM
from vllm import LLM, SamplingParams
from fastapi import FastAPI
import uvicorn

app = FastAPI(title="WallStreet Agent API", version="1.0")

# Load Quantized Model (GGUF or LoRA)
# Using GGUF format for optimized CPU/GPU offloading
MODEL_PATH = "./WallStreet_Llama_GGUF.gguf"

print(f"🚀 Loading model ecosystem from {MODEL_PATH}...")
llm = LLM(model=MODEL_PATH, quantization="gguf", dtype="half")

@app.post("/v1/generate")
async def generate_advice(query: str, max_tokens: int = 250):
    
    # Sampling parameters tuned for financial accuracy (low temp)
    sampling_params = SamplingParams(temperature=0.3, max_tokens=max_tokens)
    
    outputs = llm.generate([query], sampling_params)
    return {"response": outputs[0].outputs[0].text}

# Entrypoint
# uvicorn.run(app, host="0.0.0.0", port=8000)
"""

print("✅ Generated 'deployment.py' template. Ready for Docker encapsulation.")
print(deployment_script)


deployment_code = '''
from vllm import LLM, SamplingParams
from fastapi import FastAPI

app = FastAPI()

# 1. Load the GGUF model we just saved
llm = LLM(model="./WallStreet_Llama_GGUF.gguf", quantization="gguf")

@app.post("/generate")
def generate(query: str):
    # 2. Define Sampling (Creativity vs Precision)
    sampling_params = SamplingParams(temperature=0.3, max_tokens=200)
    
    # 3. Run High-Speed Inference
    outputs = llm.generate([query], sampling_params)
    return {"response": outputs[0].outputs[0].text}

# Run with: uvicorn main:app --host 0.0.0.0 --port 8000
'''

✅ Deployment Script Generated! (Copy this to your GitHub 'deployment.py')
